In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import rc
plt.style.use('classic')
rc('font', family='serif')
rc('figure', facecolor='w')
from astropy.io import fits, ascii
from astropy.table import Table
from collections import OrderedDict
from scipy.interpolate import UnivariateSpline
import os
import math

import TheCannon
from TheCannon import apogee, dataset, model
import apogee_tools as ap
AP_PATH = os.environ['APOGEE_DATA']

line_list = ap.lines
line_names = line_list.keys()

/home/jess/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
dfile = AP_PATH + '/training_sets/mann_bonus2/'
rfile = 'xmatch/match/apogee_x_mann.csv'

ids, wl, tr_flux, tr_ivar = apogee.load_spectra(dfile)
tr_ID = np.array([x.split('2-')[1].split('.fits')[0] for x in ids])
# tr_label = ap.loadLabels(rfile, lbl_names=['Teff', 'FeH'])
# tr_label = ap.scaleLabels(ap.loadLabels(rfile, lbl_names=['TEFF', 'FE_H']))
fopen = pd.read_csv(rfile)
tr_label = np.array([fopen['Teff'], fopen['FeH']]).T

wave = ap.rvShift(wl, rv=-80)
ds = dataset.Dataset(wave, tr_ID, tr_flux, tr_ivar, tr_label, tr_ID, tr_flux, tr_ivar)
ds.set_label_names(['T_{eff}', '[Fe/H]'])
ds.tr_label.shape, ds.tr_ID.shape, ds.tr_flux.shape, ds.tr_ivar.shape

bands = [[15140,15800],[15860,16425],[16475,16945]]
pbands = [np.where((ds.wl > bands[0][0]) & (ds.wl < bands[0][1]))[0], \
    np.where((ds.wl > bands[1][0]) & (ds.wl < bands[1][1]))[0], \
    np.where((ds.wl > bands[2][0]) & (ds.wl < bands[2][1]))[0]]

This is the right version
Loading spectra from directory /home/jess/Desktop/Research/apogee_data/training_sets/mann_bonus2/
Spectra loaded


In [ ]:
md, ds, synth = ap.fitCannonModel(ds)
np.save('bonus_cross_validation_97/model_coeff_fullset', md.coeffs)